### ANOVA for splits

In [4]:
from db_conn import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
#Sergey's output options
pd.set_option('display.max_rows', 10)
pd.set_option('precision', 3)
pd.set_option('expand_frame_repr', False)

In [5]:
split_lamoda_mobile_query = """SELECT 
splits['split_lamoda_mobile'] as split_value,dt,
sum(size(orders))/count(distinct session_id) as CR,
sum(size(orders)) as orders,
count(distinct session_id) as visits

FROM session_aggregate_new
WHERE dt between {begin_date} and {end_date} 
AND domain = 'www.lamoda.ru' AND is_office = 0 
AND depth>0

GROUP BY splits['split_lamoda_mobile'],dt"""

split_lamoda_ranking_query = """SELECT 
splits['split_lamoda_ranking'] as split_value,dt,
sum(size(orders))/count(distinct session_id) as CR,
sum(size(orders)) as orders,
count(distinct session_id) as visits

FROM session_aggregate_new
WHERE dt between {begin_date} and {end_date} 
AND domain = 'www.lamoda.ru' AND is_office = 0 
AND depth>0

GROUP BY splits['split_lamoda_ranking'],dt"""

In [6]:
#get_cursor
hive_cursor = conn_hd()

Connected to Hadoop db.


In [7]:
#execute_query
%time split_modile_df = data_hd(hive_cursor,split_lamoda_mobile_query.format(begin_date = '20150601',end_date = '20150630'))
print 'mobile ready'
%time split_ranking_df = data_hd(hive_cursor,split_lamoda_ranking_query.format(begin_date = '20150601',end_date = '20150630'))

CPU times: user 56 ms, sys: 8 ms, total: 64 ms
Wall time: 6min 49s
mobile ready
CPU times: user 116 ms, sys: 4 ms, total: 120 ms
Wall time: 1min 55s


In [13]:
import scipy.stats as st

In [37]:
split_ranking_df.to_excel('splits.xlsx')

In [20]:
split_ranking_df[split_ranking_df.split_value == 1].cr

30

In [28]:
st.f_oneway(*[split_ranking_df[split_ranking_df.split_value == i].orders for i in range(1,21)])

F_onewayResult(statistic=0.15515076620080881, pvalue=0.99998968900563789)

In [29]:
st.f_oneway(*[split_ranking_df[split_ranking_df.split_value == i].visits for i in range(1,21)])

F_onewayResult(statistic=0.0029514023007054298, pvalue=0.99999999999999989)

In [30]:
st.f_oneway(*[split_ranking_df[split_ranking_df.split_value == i].cr for i in range(1,21)])

F_onewayResult(statistic=0.29134641254140342, pvalue=0.99873427603993326)

In [31]:
st.f_oneway(*[split_modile_df[split_modile_df.split_value == i].orders for i in range(1,6)])

F_onewayResult(statistic=0.030505875123252799, pvalue=0.99819012942727303)

In [32]:
st.f_oneway(*[split_modile_df[split_modile_df.split_value == i].visits for i in range(1,6)])

F_onewayResult(statistic=0.0016988330362581235, pvalue=0.99999416192020751)

In [33]:
st.f_oneway(*[split_modile_df[split_modile_df.split_value == i].cr for i in range(1,6)])

F_onewayResult(statistic=0.076088081239541319, pvalue=0.98941552017324019)

In [34]:
#check corrctnes of list comprehansion
st.f_oneway(split_modile_df[split_modile_df.split_value == 1].visits,split_modile_df[split_modile_df.split_value == 2].visits,\
           split_modile_df[split_modile_df.split_value == 3].visits,split_modile_df[split_modile_df.split_value == 4].visits,\
           split_modile_df[split_modile_df.split_value == 5].visits)

F_onewayResult(statistic=0.0016988330362581235, pvalue=0.99999416192020751)

In [35]:
[mean(split_ranking_df[split_ranking_df.split_value == i].cr) for i in range(1,21)]

[0.018438985148329635,
 0.018183601380664237,
 0.01847882366889192,
 0.01858259119359518,
 0.01796463063646087,
 0.01825868404336788,
 0.017888342544261974,
 0.018151521969573627,
 0.018076139829457816,
 0.01822333377535806,
 0.01841225923755131,
 0.018063034268447593,
 0.018747875009470567,
 0.018503869841162885,
 0.018415938227829925,
 0.018042530681365562,
 0.01902906960275966,
 0.01818427439981813,
 0.017972183062502846,
 0.01820517994500793]